### Eff6_20fold_base (0 to 19) folds

- In this notebook we are using colab pro with high ram and 16gb GPU
- this notebook same as previus notebook 01-Eff5_20fold_base part1
- except encoder/model to EfficientNet-06

#### Important points 
- for saving outputs we need to mount drive 
- for downloading preprocessing data we need to include kaggle.json file for kaggle API
- for saving outputs we need to give output path
- output path in `Run.py` `args` class `output_dir`

In [ ]:
!nvidia-smi

Sat Nov 28 18:11:08 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    24W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# kaggle api for download preprocessed data
! mkdir /root/.kaggle
! cp '/content/drive/My Drive/kaggle.json' /root/.kaggle
! chmod 400 /root/.kaggle/kaggle.json

!pip uninstall -y kaggle >> quit
!pip install --upgrade pip >> quit
!pip install kaggle==1.5.6 >> quit
!kaggle -v >> quit

In [ ]:
!kaggle datasets download -d gopidurgaprasad/giz-nlp-agricultural-keyword-spotter
!unzip giz-nlp-agricultural-keyword-spotter.zip >> quit

 99% 569M/575M [00:17<00:00, 40.0MB/s]
100% 575M/575M [00:17<00:00, 33.7MB/s]


In [ ]:
##install requred packages
!pip -q install timm
!pip -q install albumentations
!pip -q install soundfile
!pip -q install torchlibrosa
!pip -q install audiomentations
!pip -q install catalyst
!pip -q install transformers
!pip -q install git+https://github.com/ildoonet/pytorch-gradual-warmup-lr.git

     |████████████████████████████████| 247 kB 14.2 MB/s 
     |████████████████████████████████| 631 kB 14.8 MB/s 
     |████████████████████████████████| 489 kB 12.3 MB/s 
     |████████████████████████████████| 159 kB 26.3 MB/s 
     |████████████████████████████████| 308 kB 25.4 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 1.3 MB 13.7 MB/s 
     |████████████████████████████████| 883 kB 69.3 MB/s 
     |████████████████████████████████| 1.1 MB 75.0 MB/s 
     |████████████████████████████████| 2.9 MB 82.9 MB/s 


### process data and create k-folds

In [ ]:
import glob, os, random
import pandas as pd, numpy as np
from sklearn.model_selection import StratifiedKFold

In [ ]:
train_wav = glob.glob("audio_train/input/audio_train/*/*.wav")
test_wav = glob.glob("audio_test/input/audio_test/*.wav")
print(len(train_wav), len(test_wav))

train_df = pd.DataFrame({
    "fn" : train_wav
}).sort_values("fn")
train_df["label"] = train_df.fn.apply(lambda x: x.split("/")[-2])

test_df = pd.DataFrame({
    "fn" : test_wav
}).sort_values("fn")

print(train_df.shape, test_df.shape)

4709 1017
(4709, 2) (1017, 1)


In [ ]:
FOLDS = 20
SEED = 42

train_df.loc[:, 'kfold'] = -1
train_df = train_df.sample(frac=1, random_state=SEED).reset_index(drop=True)
X = train_df['fn'].values
y = train_df['label'].values
kfold = StratifiedKFold(n_splits=FOLDS)
for fold, (t_idx, v_idx) in enumerate(kfold.split(X, y)):
    train_df.loc[v_idx, "kfold"] = fold
print(train_df.kfold.value_counts())

0     236
4     236
7     236
3     236
8     236
6     236
2     236
1     236
5     236
9     235
12    235
16    235
19    235
13    235
15    235
10    235
14    235
18    235
11    235
17    235
Name: kfold, dtype: int64


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=20.
  % (min_groups, self.n_splits)), UserWarning)


In [ ]:
train_df.to_csv("train_20folds_seed42_df.csv", index=False)
test_df.to_csv("test_df.csv", index=False)

In [ ]:
%%writefile Codes.py
CODE = {
 'Pump': 0,
 'Spinach': 1,
 'abalimi': 2,
 'afukirira': 3,
 'agriculture': 4,
 'akammwanyi': 5,
 'akamonde': 6,
 'akasaanyi': 7,
 'akatunda': 8,
 'akatungulu': 9,
 'akawuka': 10,
 'amakoola': 11,
 'amakungula': 12,
 'amalagala': 13,
 'amappapaali': 14,
 'amatooke': 15,
 'banana': 16,
 'beans': 17,
 'bibala': 18,
 'bulimi': 19,
 'butterfly': 20,
 'cabbages': 21,
 'cassava': 22,
 'caterpillar': 23,
 'caterpillars': 24,
 'coffee': 25,
 'crop': 26,
 'ddagala': 27,
 'dig': 28,
 'disease': 29,
 'doodo': 30,
 'drought': 31,
 'ebbugga': 32,
 'ebibala': 33,
 'ebigimusa': 34,
 'ebijanjaalo': 35,
 'ebijjanjalo': 36,
 'ebikajjo': 37,
 'ebikolo': 38,
 'ebikongoliro': 39,
 'ebikoola': 40,
 'ebimera': 41,
 'ebinyebwa': 42,
 'ebirime': 43,
 'ebisaanyi': 44,
 'ebisooli': 45,
 'ebisoolisooli': 46,
 'ebitooke': 47,
 'ebiwojjolo': 48,
 'ebiwuka': 49,
 'ebyobulimi': 50,
 'eddagala': 51,
 'eggobe': 52,
 'ejjobyo': 53,
 'ekibala': 54,
 'ekigimusa': 55,
 'ekijanjaalo': 56,
 'ekikajjo': 57,
 'ekikolo': 58,
 'ekikoola': 59,
 'ekimera': 60,
 'ekirime': 61,
 'ekirwadde': 62,
 'ekisaanyi': 63,
 'ekitooke': 64,
 'ekiwojjolo': 65,
 'ekyeya': 66,
 'emboga': 67,
 'emicungwa': 68,
 'emisiri': 69,
 'emiyembe': 70,
 'emmwanyi': 71,
 'endagala': 72,
 'endokwa': 73,
 'endwadde': 74,
 'enkota': 75,
 'ennima': 76,
 'ennimiro': 77,
 'ennyaanya': 78,
 'ensigo': 79,
 'ensiringanyi': 80,
 'ensujju': 81,
 'ensuku': 82,
 'ensukusa': 83,
 'enva endiirwa': 84,
 'eppapaali': 85,
 'faamu': 86,
 'farm': 87,
 'farmer': 88,
 'farming instructor': 89,
 'fertilizer': 90,
 'fruit': 91,
 'fruit picking': 92,
 'garden': 93,
 'greens': 94,
 'ground nuts': 95,
 'harvest': 96,
 'harvesting': 97,
 'insect': 98,
 'insects': 99,
 'irish potatoes': 100,
 'irrigate': 101,
 'kaamulali': 102,
 'kasaanyi': 103,
 'kassooli': 104,
 'kikajjo': 105,
 'kikolo': 106,
 'kisaanyi': 107,
 'kukungula': 108,
 'leaf': 109,
 'leaves': 110,
 'lumonde': 111,
 'lusuku': 112,
 'maize': 113,
 'maize stalk borer': 114,
 'maize streak virus': 115,
 'mango': 116,
 'mangoes': 117,
 'matooke': 118,
 'matooke seedlings': 119,
 'medicine': 120,
 'miceere': 121,
 'micungwa': 122,
 'mpeke': 123,
 'muceere': 124,
 'mucungwa': 125,
 'mulimi': 126,
 'munyeera': 127,
 'muwogo': 128,
 'nakavundira': 129,
 'nambaale': 130,
 'namuginga': 131,
 'ndwadde': 132,
 'nfukirira': 133,
 'nnakati': 134,
 'nnasale beedi': 135,
 'nnimiro': 136,
 'nnyaanya': 137,
 'npk': 138,
 'nursery bed': 139,
 'obulimi': 140,
 'obulwadde': 141,
 'obumonde': 142,
 'obusaanyi': 143,
 'obutunda': 144,
 'obutungulu': 145,
 'obuwuka': 146,
 'okufukirira': 147,
 'okufuuyira': 148,
 'okugimusa': 149,
 'okukkoola': 150,
 'okukungula': 151,
 'okulima': 152,
 'okulimibwa': 153,
 'okunnoga': 154,
 'okusaasaana': 155,
 'okusaasaanya': 156,
 'okusiga': 157,
 'okusimba': 158,
 'okuzifuuyira': 159,
 'olusuku': 160,
 'omuceere': 161,
 'omucungwa': 162,
 'omulimi': 163,
 'omulimisa': 164,
 'omusiri': 165,
 'omuyembe': 166,
 'onion': 167,
 'orange': 168,
 'pampu': 169,
 'passion fruit': 170,
 'pawpaw': 171,
 'pepper': 172,
 'plant': 173,
 'plantation': 174,
 'ppaapaali': 175,
 'pumpkin': 176,
 'rice': 177,
 'seed': 178,
 'sikungula': 179,
 'sow': 180,
 'spray': 181,
 'spread': 182,
 'suckers': 183,
 'sugarcane': 184,
 'sukumawiki': 185,
 'super grow': 186,
 'sweet potatoes': 187,
 'tomatoes': 188,
 'vegetables': 189,
 'watermelon': 190,
 'weeding': 191,
 'worm': 192
}

INV_CODE = {v: k for k, v in CODE.items()}

Writing Codes.py


In [ ]:
%%writefile pytorch_utils.py
import numpy as np
import time
import torch
import torch.nn as nn


def move_data_to_device(x, device):
    if 'float' in str(x.dtype):
        x = torch.Tensor(x)
    elif 'int' in str(x.dtype):
        x = torch.LongTensor(x)
    else:
        return x

    return x.to(device)


def do_mixup(x, mixup_lambda):
    """Mixup x of even indexes (0, 2, 4, ...) with x of odd indexes 
    (1, 3, 5, ...).

    Args:
      x: (batch_size * 2, ...)
      mixup_lambda: (batch_size * 2,)

    Returns:
      out: (batch_size, ...)
    """
    out = (x[0 :: 2].transpose(0, -1) * mixup_lambda[0 :: 2] + \
        x[1 :: 2].transpose(0, -1) * mixup_lambda[1 :: 2]).transpose(0, -1)
    return out
    

def append_to_dict(dict, key, value):
    if key in dict.keys():
        dict[key].append(value)
    else:
        dict[key] = [value]


def forward(model, generator, return_input=False, 
    return_target=False):
    """Forward data to a model.
    
    Args: 
      model: object
      generator: object
      return_input: bool
      return_target: bool

    Returns:
      audio_name: (audios_num,)
      clipwise_output: (audios_num, classes_num)
      (ifexist) segmentwise_output: (audios_num, segments_num, classes_num)
      (ifexist) framewise_output: (audios_num, frames_num, classes_num)
      (optional) return_input: (audios_num, segment_samples)
      (optional) return_target: (audios_num, classes_num)
    """
    output_dict = {}
    device = next(model.parameters()).device
    time1 = time.time()

    # Forward data to a model in mini-batches
    for n, batch_data_dict in enumerate(generator):
        print(n)
        batch_waveform = move_data_to_device(batch_data_dict['waveform'], device)
        
        with torch.no_grad():
            model.eval()
            batch_output = model(batch_waveform)

        append_to_dict(output_dict, 'audio_name', batch_data_dict['audio_name'])

        append_to_dict(output_dict, 'clipwise_output', 
            batch_output['clipwise_output'].data.cpu().numpy())

        if 'segmentwise_output' in batch_output.keys():
            append_to_dict(output_dict, 'segmentwise_output', 
                batch_output['segmentwise_output'].data.cpu().numpy())

        if 'framewise_output' in batch_output.keys():
            append_to_dict(output_dict, 'framewise_output', 
                batch_output['framewise_output'].data.cpu().numpy())
            
        if return_input:
            append_to_dict(output_dict, 'waveform', batch_data_dict['waveform'])
            
        if return_target:
            if 'target' in batch_data_dict.keys():
                append_to_dict(output_dict, 'target', batch_data_dict['target'])

        if n % 10 == 0:
            print(' --- Inference time: {:.3f} s / 10 iterations ---'.format(
                time.time() - time1))
            time1 = time.time()

    for key in output_dict.keys():
        output_dict[key] = np.concatenate(output_dict[key], axis=0)

    return output_dict


def interpolate(x, ratio):
    """Interpolate data in time domain. This is used to compensate the 
    resolution reduction in downsampling of a CNN.
    
    Args:
      x: (batch_size, time_steps, classes_num)
      ratio: int, ratio to interpolate

    Returns:
      upsampled: (batch_size, time_steps * ratio, classes_num)
    """
    (batch_size, time_steps, classes_num) = x.shape
    upsampled = x[:, :, None, :].repeat(1, 1, ratio, 1)
    upsampled = upsampled.reshape(batch_size, time_steps * ratio, classes_num)
    return upsampled


def pad_framewise_output(framewise_output, frames_num):
    """Pad framewise_output to the same length as input frames. The pad value 
    is the same as the value of the last frame.

    Args:
      framewise_output: (batch_size, frames_num, classes_num)
      frames_num: int, number of frames to pad

    Outputs:
      output: (batch_size, frames_num, classes_num)
    """
    pad = framewise_output[:, -1 :, :].repeat(1, frames_num - framewise_output.shape[1], 1)
    """tensor for padding"""

    output = torch.cat((framewise_output, pad), dim=1)
    """(batch_size, frames_num, classes_num)"""

    return output


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


def count_flops(model, audio_length):
    """Count flops. Code modified from others' implementation.
    """
    multiply_adds = True
    list_conv2d=[]
    def conv2d_hook(self, input, output):
        batch_size, input_channels, input_height, input_width = input[0].size()
        output_channels, output_height, output_width = output[0].size()
 
        kernel_ops = self.kernel_size[0] * self.kernel_size[1] * (self.in_channels / self.groups) * (2 if multiply_adds else 1)
        bias_ops = 1 if self.bias is not None else 0
 
        params = output_channels * (kernel_ops + bias_ops)
        flops = batch_size * params * output_height * output_width
 
        list_conv2d.append(flops)

    list_conv1d=[]
    def conv1d_hook(self, input, output):
        batch_size, input_channels, input_length = input[0].size()
        output_channels, output_length = output[0].size()
 
        kernel_ops = self.kernel_size[0] * (self.in_channels / self.groups) * (2 if multiply_adds else 1)
        bias_ops = 1 if self.bias is not None else 0
 
        params = output_channels * (kernel_ops + bias_ops)
        flops = batch_size * params * output_length
 
        list_conv1d.append(flops)
 
    list_linear=[] 
    def linear_hook(self, input, output):
        batch_size = input[0].size(0) if input[0].dim() == 2 else 1
 
        weight_ops = self.weight.nelement() * (2 if multiply_adds else 1)
        bias_ops = self.bias.nelement()
 
        flops = batch_size * (weight_ops + bias_ops)
        list_linear.append(flops)
 
    list_bn=[] 
    def bn_hook(self, input, output):
        list_bn.append(input[0].nelement() * 2)
 
    list_relu=[] 
    def relu_hook(self, input, output):
        list_relu.append(input[0].nelement() * 2)
 
    list_pooling2d=[]
    def pooling2d_hook(self, input, output):
        batch_size, input_channels, input_height, input_width = input[0].size()
        output_channels, output_height, output_width = output[0].size()
 
        kernel_ops = self.kernel_size * self.kernel_size
        bias_ops = 0
        params = output_channels * (kernel_ops + bias_ops)
        flops = batch_size * params * output_height * output_width
 
        list_pooling2d.append(flops)

    list_pooling1d=[]
    def pooling1d_hook(self, input, output):
        batch_size, input_channels, input_length = input[0].size()
        output_channels, output_length = output[0].size()
 
        kernel_ops = self.kernel_size[0]
        bias_ops = 0
        
        params = output_channels * (kernel_ops + bias_ops)
        flops = batch_size * params * output_length
 
        list_pooling2d.append(flops)
 
    def foo(net):
        childrens = list(net.children())
        if not childrens:
            if isinstance(net, nn.Conv2d):
                net.register_forward_hook(conv2d_hook)
            elif isinstance(net, nn.Conv1d):
                net.register_forward_hook(conv1d_hook)
            elif isinstance(net, nn.Linear):
                net.register_forward_hook(linear_hook)
            elif isinstance(net, nn.BatchNorm2d) or isinstance(net, nn.BatchNorm1d):
                net.register_forward_hook(bn_hook)
            elif isinstance(net, nn.ReLU):
                net.register_forward_hook(relu_hook)
            elif isinstance(net, nn.AvgPool2d) or isinstance(net, nn.MaxPool2d):
                net.register_forward_hook(pooling2d_hook)
            elif isinstance(net, nn.AvgPool1d) or isinstance(net, nn.MaxPool1d):
                net.register_forward_hook(pooling1d_hook)
            else:
                print('Warning: flop of module {} is not counted!'.format(net))
            return
        for c in childrens:
            foo(c)

    # Register hook
    foo(model)
    
    device = device = next(model.parameters()).device
    input = torch.rand(1, audio_length).to(device)

    out = model(input)
 
    total_flops = sum(list_conv2d) + sum(list_conv1d) + sum(list_linear) + \
        sum(list_bn) + sum(list_relu) + sum(list_pooling2d) + sum(list_pooling1d)
    
    return total_flops


Writing pytorch_utils.py


In [ ]:
%%writefile Datasets.py
import random, glob
import numpy as np, pandas as pd
import soundfile as sf

import torch
from torch.utils.data import Dataset
from albumentations.pytorch.functional import img_to_tensor

from Codes import CODE, INV_CODE

class AudioDataset(Dataset):
    def __init__(self, df, period=1, transforms=None, train=True):
        
        self.period = period
        self.transforms = transforms
        self.train = train

        self.wav_paths = df["fn"].values
        if train:
            self.labels = df["label"].values
        else:
            self.labels = np.zeros_like(self.wav_paths)
    
    def __len__(self):
        return len(self.wav_paths)
    
    def __getitem__(self, idx):
        wav_path, code = self.wav_paths[idx], self.labels[idx]
        label = np.zeros(len(CODE), dtype='f')

        y, sr = sf.read(wav_path)

        if self.transforms:
            y = self.transforms(samples=y, sample_rate=sr)
        
        len_y = len(y)
        effective_length = sr * self.period
        if len_y < effective_length:
            new_y = np.zeros(effective_length, dtype=y.dtype)
            start = np.random.randint(effective_length - len_y)
            new_y[start:start+len_y] = y
            y = new_y#.astype(np.float)
        elif len_y > effective_length:
            start = np.random.randint(len_y - effective_length)
            y = y[start:start + effective_length]#.astype(np.float32)
        else:
            y = y#.astype(np.float32)

        if self.train:
            #label[CODE[code]] = 1
            label = CODE[code]
        else:
            label = 0

        return {
            "waveform" : y, #torch.tensor(y, dtype=torch.double),
            "target" : torch.tensor(label, dtype=torch.long)
        }
    
    def __get_labels__(self):
        return self.labels

Writing Datasets.py


In [ ]:
%%writefile Augmentation.py
import audiomentations as A

augmenter = A.Compose([
    A.AddGaussianNoise(p=0.4),
    A.AddGaussianSNR(p=0.4),
    #A.AddBackgroundNoise("../input/train_audio/", p=1)
    #A.AddImpulseResponse(p=0.1),
    #A.AddShortNoises("../input/train_audio/", p=1)
    A.FrequencyMask(min_frequency_band=0.0,  max_frequency_band=0.2, p=0.05),
    A.TimeMask(min_band_part=0.0, max_band_part=0.2, p=0.05),
    A.PitchShift(min_semitones=-0.5, max_semitones=0.5, p=0.05),
    A.Shift(p=0.1),
    A.Normalize(p=0.1),
    A.ClippingDistortion(min_percentile_threshold=0, max_percentile_threshold=1, p=0.05),
    A.PolarityInversion(p=0.05),
    A.Gain(p=0.2)
])

test_augmenter = A.Compose([
    A.AddGaussianNoise(p=0.3),
    A.AddGaussianSNR(p=0.3),
    #A.AddBackgroundNoise("../input/train_audio/", p=1)
    #A.AddImpulseResponse(p=0.1),
    #A.AddShortNoises("../input/train_audio/", p=1)
    A.FrequencyMask(min_frequency_band=0.0,  max_frequency_band=0.2, p=0.05),
    A.TimeMask(min_band_part=0.0, max_band_part=0.2, p=0.05),
    A.PitchShift(min_semitones=-0.5, max_semitones=0.5, p=0.05),
    A.Shift(p=0.1),
    A.Normalize(p=0.1),
    A.ClippingDistortion(min_percentile_threshold=0, max_percentile_threshold=1, p=0.05),
    A.PolarityInversion(p=0.05),
    A.Gain(p=0.1)
])




Writing Augmentation.py


In [ ]:
%%writefile Models.py
import numpy as np
from functools import partial

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules.dropout import Dropout
from torch.nn.modules.linear import Linear
from torch.nn.modules.pooling import AdaptiveAvgPool2d, AdaptiveMaxPool2d

import timm
from timm.models.efficientnet import tf_efficientnet_b4_ns, tf_efficientnet_b3_ns, \
    tf_efficientnet_b5_ns, tf_efficientnet_b2_ns, tf_efficientnet_b6_ns, tf_efficientnet_b7_ns, tf_efficientnet_b0_ns

from torchlibrosa.stft import Spectrogram, LogmelFilterBank
from torchlibrosa.augmentation import SpecAugmentation

from pytorch_utils import do_mixup, interpolate, pad_framewise_output

encoder_params = {
    "resnest50d" : {
        "features" : 2048,
        "init_op"  : partial(timm.models.resnest50d, pretrained=True, in_chans=1)
    },
    "densenet201" : {
        "features": 1920,
        "init_op": partial(timm.models.densenet201, pretrained=True)
    },
    "dpn92" : {
        "features": 2688,
        "init_op": partial(timm.models.dpn92, pretrained=True)
    },
    "dpn131": {
        "features": 2688,
        "init_op": partial(timm.models.dpn131, pretrained=True)
    },
    "tf_efficientnet_b0_ns": {
        "features": 1280,
        "init_op": partial(tf_efficientnet_b0_ns, pretrained=True, drop_path_rate=0.2, in_chans=1)
    },
    "tf_efficientnet_b3_ns": {
        "features": 1536,
        "init_op": partial(tf_efficientnet_b3_ns, pretrained=True, drop_path_rate=0.2, in_chans=1)
    },
    "tf_efficientnet_b2_ns": {
        "features": 1408,
        "init_op": partial(tf_efficientnet_b2_ns, pretrained=True, drop_path_rate=0.2, in_chans=1)
    },
    "tf_efficientnet_b4_ns": {
        "features": 1792,
        "init_op": partial(tf_efficientnet_b4_ns, pretrained=True, drop_path_rate=0.2, in_chans=1)
    },
    "tf_efficientnet_b5_ns": {
        "features": 2048,
        "init_op": partial(tf_efficientnet_b5_ns, pretrained=True, drop_path_rate=0.2, in_chans=1)
    },
    "tf_efficientnet_b6_ns": {
        "features": 2304,
        "init_op": partial(tf_efficientnet_b6_ns, pretrained=True, drop_path_rate=0.2, in_chans=1)
    },
}


class AudioClassifier(nn.Module):
    def __init__(self, encoder, sample_rate, window_size, hop_size, mel_bins, fmin, fmax, classes_num):
        super().__init__()

        window = 'hann'
        center = True
        pad_mode = 'reflect'
        ref = 1.0
        amin = 1e-10
        top_db = None

        # Spectrogram extractor
        self.spectrogram_extractor = Spectrogram(n_fft=window_size, hop_length=hop_size, 
            win_length=window_size, window=window, center=center, pad_mode=pad_mode, 
            freeze_parameters=True)

        # Logmel feature extractor
        self.logmel_extractor = LogmelFilterBank(sr=sample_rate, n_fft=window_size, 
            n_mels=mel_bins, fmin=fmin, fmax=fmax, ref=ref, amin=amin, top_db=top_db, 
            freeze_parameters=True)

        # Spec augmenter
        self.spec_augmenter = SpecAugmentation(time_drop_width=64, time_stripes_num=2, 
            freq_drop_width=8, freq_stripes_num=2)
        
        self.encoder = encoder_params[encoder]["init_op"]()
        self.avg_pool = AdaptiveAvgPool2d((1, 1))
        self.dropout = Dropout(0.3)
        self.fc = Linear(encoder_params[encoder]['features'], classes_num)
    
    def forward(self, input, spec_aug=False, mixup_lambda=None):
        #print(input.type())
        x = self.spectrogram_extractor(input.float()) # (batch_size, 1, time_steps, freq_bins)
        x = self.logmel_extractor(x) # (batch_size, 1, time_steps, mel_bins)

        #if spec_aug:
        #    x = self.spec_augmenter(x)
        if self.training:
            x = self.spec_augmenter(x)
        
        # Mixup on spectrogram
        if mixup_lambda is not None:
            x = do_mixup(x, mixup_lambda)
            #pass
        
        x = self.encoder.forward_features(x)
        x = self.avg_pool(x).flatten(1)
        x = self.dropout(x)
        x = self.fc(x)
        return x


def init_layer(layer):
    """Initialize a Linear or Convolutional layer. """
    nn.init.xavier_uniform_(layer.weight)
 
    if hasattr(layer, 'bias'):
        if layer.bias is not None:
            layer.bias.data.fill_(0.)
            
    
def init_bn(bn):
    """Initialize a Batchnorm layer. """
    bn.bias.data.fill_(0.)
    bn.weight.data.fill_(1.)


class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        
        super(ConvBlock, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=in_channels, 
                              out_channels=out_channels,
                              kernel_size=(3, 3), stride=(1, 1),
                              padding=(1, 1), bias=False)
                              
        self.conv2 = nn.Conv2d(in_channels=out_channels, 
                              out_channels=out_channels,
                              kernel_size=(3, 3), stride=(1, 1),
                              padding=(1, 1), bias=False)
                              
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.init_weight()
        
    def init_weight(self):
        init_layer(self.conv1)
        init_layer(self.conv2)
        init_bn(self.bn1)
        init_bn(self.bn2)

        
    def forward(self, input, pool_size=(2, 2), pool_type='avg'):
        
        x = input
        x = F.relu_(self.bn1(self.conv1(x)))
        x = F.relu_(self.bn2(self.conv2(x)))
        if pool_type == 'max':
            x = F.max_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg':
            x = F.avg_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg+max':
            x1 = F.avg_pool2d(x, kernel_size=pool_size)
            x2 = F.max_pool2d(x, kernel_size=pool_size)
            x = x1 + x2
        else:
            raise Exception('Incorrect argument!')
        
        return x


class ConvBlock5x5(nn.Module):
    def __init__(self, in_channels, out_channels):
        
        super(ConvBlock5x5, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=in_channels, 
                              out_channels=out_channels,
                              kernel_size=(5, 5), stride=(1, 1),
                              padding=(2, 2), bias=False)
                              
        self.bn1 = nn.BatchNorm2d(out_channels)

        self.init_weight()
        
    def init_weight(self):
        init_layer(self.conv1)
        init_bn(self.bn1)

        
    def forward(self, input, pool_size=(2, 2), pool_type='avg'):
        
        x = input
        x = F.relu_(self.bn1(self.conv1(x)))
        if pool_type == 'max':
            x = F.max_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg':
            x = F.avg_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg+max':
            x1 = F.avg_pool2d(x, kernel_size=pool_size)
            x2 = F.max_pool2d(x, kernel_size=pool_size)
            x = x1 + x2
        else:
            raise Exception('Incorrect argument!')
        
        return x


class AttBlock(nn.Module):
    def __init__(self, n_in, n_out, activation='linear', temperature=1.):
        super(AttBlock, self).__init__()
        
        self.activation = activation
        self.temperature = temperature
        self.att = nn.Conv1d(in_channels=n_in, out_channels=n_out, kernel_size=1, stride=1, padding=0, bias=True)
        self.cla = nn.Conv1d(in_channels=n_in, out_channels=n_out, kernel_size=1, stride=1, padding=0, bias=True)
        
        self.bn_att = nn.BatchNorm1d(n_out)
        self.init_weights()
        
    def init_weights(self):
        init_layer(self.att)
        init_layer(self.cla)
        init_bn(self.bn_att)
         
    def forward(self, x):
        # x: (n_samples, n_in, n_time)
        norm_att = torch.softmax(torch.clamp(self.att(x), -10, 10), dim=-1)
        cla = self.nonlinear_transform(self.cla(x))
        x = torch.sum(norm_att * cla, dim=2)
        return x, norm_att, cla

    def nonlinear_transform(self, x):
        if self.activation == 'linear':
            return x
        elif self.activation == 'sigmoid':
            return torch.sigmoid(x)

class Cnn14(nn.Module):
    def __init__(self, sample_rate, window_size, hop_size, mel_bins, fmin, 
        fmax, classes_num):
        
        super(Cnn14, self).__init__()

        window = 'hann'
        center = True
        pad_mode = 'reflect'
        ref = 1.0
        amin = 1e-10
        top_db = None

        # Spectrogram extractor
        self.spectrogram_extractor = Spectrogram(n_fft=window_size, hop_length=hop_size, 
            win_length=window_size, window=window, center=center, pad_mode=pad_mode, 
            freeze_parameters=True)

        # Logmel feature extractor
        self.logmel_extractor = LogmelFilterBank(sr=sample_rate, n_fft=window_size, 
            n_mels=mel_bins, fmin=fmin, fmax=fmax, ref=ref, amin=amin, top_db=top_db, 
            freeze_parameters=True)

        # Spec augmenter
        self.spec_augmenter = SpecAugmentation(time_drop_width=64, time_stripes_num=2, 
            freq_drop_width=8, freq_stripes_num=2)

        self.bn0 = nn.BatchNorm2d(64)

        self.conv_block1 = ConvBlock(in_channels=1, out_channels=64)
        self.conv_block2 = ConvBlock(in_channels=64, out_channels=128)
        self.conv_block3 = ConvBlock(in_channels=128, out_channels=256)
        self.conv_block4 = ConvBlock(in_channels=256, out_channels=512)
        self.conv_block5 = ConvBlock(in_channels=512, out_channels=1024)
        self.conv_block6 = ConvBlock(in_channels=1024, out_channels=2048)

        self.fc1 = nn.Linear(2048, 2048, bias=True)
        self.fc_audioset1 = nn.Linear(2048, classes_num, bias=True)
        
        self.init_weight()

    def init_weight(self):
        init_bn(self.bn0)
        init_layer(self.fc1)
        init_layer(self.fc_audioset1)
 
    def forward(self, input, mixup_lambda=None):
        """
        Input: (batch_size, data_length)"""

        x = self.spectrogram_extractor(input.float())   # (batch_size, 1, time_steps, freq_bins)
        x = self.logmel_extractor(x)    # (batch_size, 1, time_steps, mel_bins)

        x = x.transpose(1, 3)
        x = self.bn0(x)
        x = x.transpose(1, 3)
        
        if self.training:
            x = self.spec_augmenter(x)

        # Mixup on spectrogram
        if self.training and mixup_lambda is not None:
            x = do_mixup(x, mixup_lambda)

        x = self.conv_block1(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block2(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block3(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block4(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block5(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block6(x, pool_size=(1, 1), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = torch.mean(x, dim=3)
        
        (x1, _) = torch.max(x, dim=2)
        x2 = torch.mean(x, dim=2)
        x = x1 + x2
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu_(self.fc1(x))
        #embedding = F.dropout(x, p=0.5, training=self.training)
        #clipwise_output = torch.sigmoid(self.fc_audioset(x))
        x = self.fc_audioset1(x)
        
        #output_dict = {'clipwise_output': clipwise_output, 'embedding': embedding}

        return x



Writing Models.py


In [ ]:
%%writefile Utils.py
import torch
import numpy as np
from sklearn import metrics
from sklearn.metrics import log_loss

def logloss_metric(y_true, y_pred):
    y_true = np.asarray(y_true).ravel()
    y_pred = np.asarray(y_pred).ravel()
    y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
    loss = np.where(y_true == 1, -np.log(y_pred), -np.log(1 - y_pred))
    return loss.mean()

class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

class MetricMeter(object):
    def __init__(self):
        self.reset()
    
    def reset(self):
        self.y_true = []
        self.y_pred = []
    
    def update(self, y_true, y_pred):
        self.y_true.extend(y_true.cpu().detach().numpy().tolist())
        self.y_pred.extend(torch.nn.functional.softmax(y_pred).cpu().detach().numpy().tolist())

    @property
    def avg(self):
        #self.logloss = torch.nn.CrossEntropyLoss()(torch.tensor(self.y_pred), torch.tensor(self.y_true)).item()#np.argmax(self.y_true, axis=1)
        self.logloss = log_loss(self.y_true, self.y_pred, labels=range(0, 193))
        self.acc = metrics.accuracy_score(self.y_true, np.argmax(self.y_pred, axis=1))
        self.f1 = metrics.f1_score(self.y_true, np.argmax(self.y_pred, axis=1), labels=range(0, 193), average="micro")
    
        return {
            "logloss" : self.logloss,
            "acc" : self.acc,
            "f1" : self.f1

        }

Writing Utils.py


In [ ]:
%%writefile Losses.py
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss

Writing Losses.py


In [ ]:
%%writefile Functions.py
from tqdm import tqdm

import numpy as np
import torch, torch.nn as nn
import torch.nn.functional as F

from Utils import AverageMeter, MetricMeter

def train_epoch(args, model, loader, criterion, optimizer, scheduler, epoch):
    losses = AverageMeter()
    scores = MetricMeter()

    model.train()
    #scaler = torch.cuda.amp.GradScaler()

    t = tqdm(loader)
    for i, sample in enumerate(t):
        optimizer.zero_grad()
        input = sample['waveform'].to(args.device)
        target = sample['target'].to(args.device)
        #print(input.shape)
        #with torch.cuda.amp.autocast(enabled=args.amp):
        output = model(input)
        loss = criterion(output, target)
        #scaler.scale(loss).backward()
        #scaler.step(optimizer)
        #scaler.update()
        loss.backward()
        optimizer.step()
        if scheduler and args.step_scheduler:
            scheduler.step()

        bs = input.size(0)
        scores.update(target, output)
        losses.update(loss.item(), bs)

        t.set_description(f"Train E:{epoch} - Loss{losses.avg:0.4f}")
    t.close()
    return scores.avg, losses.avg

def valid_epoch(args, model, loader, criterion, epoch):
    losses = AverageMeter()
    scores = MetricMeter()

    model.eval()

    with torch.no_grad():
        t = tqdm(loader)
        for i, sample in enumerate(t):
            input = sample['waveform'].to(args.device)
            target = sample['target'].to(args.device)
            output = model(input)
            loss = criterion(output, target)

            bs = input.size(0)
            scores.update(target, output)
            losses.update(loss.item(), bs)
            t.set_description(f"Valid E:{epoch} - Loss:{losses.avg:0.4f}")
    t.close()
    return scores.avg, losses.avg

def test_epoch(args, model, loader):
    model.eval()
    pred_list = []
    with torch.no_grad():
        t = tqdm(loader)
        for i, sample in enumerate(t):
            input = sample["waveform"].to(args.device)
            output = torch.nn.Softmax()(model(input)).cpu().detach().numpy().tolist()
            pred_list.extend(output)
    
    return pred_list

def TTA_epoch(args, model, loader, ntta=10):
    tta_preds = []
    for i in range(ntta):
        model.eval()
        pred_list = []
        with torch.no_grad():
            t = tqdm(loader)
            for i, sample in enumerate(t):
                input = sample["waveform"].to(args.device)
                output = torch.nn.Softmax()(model(input)).cpu().detach().numpy().tolist()
                pred_list.extend(output)
        tta_preds.append(pred_list)
    return np.mean(tta_preds, axis=0)

Writing Functions.py


In [ ]:
%%writefile Run.py
import warnings
warnings.filterwarnings('ignore')

import os, time, librosa, random
import numpy as np, pandas as pd

import torch, torch.nn as nn
import torch.nn.functional as F

from transformers import get_linear_schedule_with_warmup
from catalyst.data.sampler import DistributedSampler, BalanceClassSampler
from tqdm import tqdm

try:
    import wandb
except:
    wandb = False

import Codes
import Datasets
import Models
import Losses
import Functions
import Augmentation

class args:
    DEBUG = False
    amp = False
    wandb = False
    exp_name = "Eff6_20fold_base"
    network = "AudioClassifier" #"Cnn14" #"AudioClassifier"
    encoder = "ResNet38"
    pretrain_weights = None #"/content/Cnn14_mAP=0.431.pth"
    model_param = {
        'encoder' : 'tf_efficientnet_b6_ns',
        'sample_rate': 32000,
        'window_size' : 1024,
        'hop_size' : 320,
        'mel_bins' : 64,
        'fmin' : 50,
        'fmax' : 14000,
        'classes_num' : 193 
    }
    losses = "CrossEntropyLoss" #"BCEWithLogitsLoss"
    lr = 1e-3
    step_scheduler = True
    epoch_scheduler = False
    period = 3
    seed = 42
    start_epoch = 0
    epochs = 50
    batch_size = 64
    num_workers = 2
    early_stop = 10

    device = ('cuda' if torch.cuda.is_available() else 'cpu')
    train_csv = "train_20folds_seed42_df.csv"
    test_csv = "test_df.csv"
    sub_csv = "SampleSubmission.csv"
    output_dir = "/content/drive/MyDrive/ZINDI GIZ NLP Agricultural Keyword Spotter #3 place solution/weights"

def main(fold):

    # Setting seed
    seed = args.seed
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

    args.fold = fold
    args.save_path = os.path.join(args.output_dir, args.exp_name)
    os.makedirs(args.save_path, exist_ok=True)

    train_df = pd.read_csv(args.train_csv)
    test_df = pd.read_csv(args.test_csv)
    sub_df = pd.read_csv(args.sub_csv)
    if args.DEBUG:
        train_df = train_df.sample(1000)
    train_fold = train_df[train_df.kfold != fold]
    valid_fold = train_df[train_df.kfold == fold]

    train_dataset = Datasets.AudioDataset(
        df=train_fold,
        period=args.period,
        transforms=Augmentation.augmenter,
        train=True
    )
    valid_dataset = Datasets.AudioDataset(
        df=valid_fold,
        period=args.period,
        transforms=None,
        train=True
    )
    test_dataset = Datasets.AudioDataset(
        df=test_df,
        period=args.period,
        transforms=None,
        train=False
    )

    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=args.batch_size,
        #sampler = BalanceClassSampler(labels=train_dataset.__get_labels__(), mode="upsampling"),
        shuffle=True,
        drop_last=True,
        num_workers=args.num_workers
    )
    valid_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=args.batch_size,
        shuffle=False,
        drop_last=False,
        num_workers=args.num_workers
    )
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=args.batch_size,
        shuffle=False,
        drop_last=False,
        num_workers=args.num_workers
    )

    tta_dataset = Datasets.AudioDataset(
        df=test_df,
        period=args.period,
        transforms=Augmentation.test_augmenter,
        train=False
    )
    tta_loader = torch.utils.data.DataLoader(
        tta_dataset,
        batch_size=args.batch_size,
        shuffle=False,
        drop_last=False,
        num_workers=args.num_workers
    )

    model = Models.__dict__[args.network](**args.model_param)
    model = model.to(args.device)

    if args.pretrain_weights:
        print("---------------------loading pretrain weights")
        model.load_state_dict(torch.load(args.pretrain_weights, map_location=args.device)["model"], strict=False)
        model = model.to(args.device)

    criterion = Losses.__dict__[args.losses]()
    optimizer = torch.optim.AdamW(model.parameters(), lr=args.lr)
    num_train_steps = int(len(train_loader) * args.epochs)
    num_warmup_steps = int(0.1 * args.epochs * len(train_loader))
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_train_steps)
    
    best_logloss = np.inf
    for epoch in range(args.start_epoch, args.epochs):
        train_avg, train_loss = Functions.train_epoch(args, model, train_loader, criterion, optimizer, scheduler, epoch)
        valid_avg, valid_loss = Functions.valid_epoch(args, model, valid_loader, criterion, epoch)
        
        if args.epoch_scheduler:
            scheduler.step()

        content = f"""
                {time.ctime()} \n
                Fold:{args.fold}, Epoch:{epoch}, lr:{optimizer.param_groups[0]['lr']:.7}\n
                Train Loss:{train_loss:0.4f} - LogLoss:{train_avg['logloss']:0.4f} --- ACC:{train_avg['acc']:0.4f} --- F1:{train_avg['f1']:0.4f}\n
                Valid Loss:{valid_loss:0.4f} - LogLoss:{valid_avg['logloss']:0.4f} --- ACC:{valid_avg['acc']:0.4f} --- F1:{valid_avg['f1']:0.4f}\n
        """
        print(content)
        with open(f'{args.save_path}/log_{args.exp_name}.txt', 'a') as appender:
            appender.write(content+'\n')
        
        if valid_avg['logloss'] < best_logloss:
            print(f"########## >>>>>>>> Model Improved From {best_logloss} ----> {valid_avg['logloss']}")
            torch.save(model.state_dict(), os.path.join(args.save_path, f'fold-{args.fold}.bin'))
            best_logloss = valid_avg['logloss']
        #torch.save(model.state_dict(), os.path.join(args.save_path, f'fold-{args.fold}_last.bin'))

    model.load_state_dict(torch.load(os.path.join(args.save_path, f'fold-{args.fold}.bin'), map_location=args.device))
    model = model.to(args.device)

    target_cols = sub_df.columns.values.tolist()
    test_pred = Functions.test_epoch(args, model, test_loader)
    print(np.array(test_pred).shape)
    tta_pred = Functions.TTA_epoch(args, model, tta_loader, ntta=10)
    print(np.array(tta_pred).shape)
    
    test_pred_df = pd.DataFrame({
        "fn" : test_df.fn.values
    })
    test_pred_df["fn"] = test_pred_df["fn"].apply(lambda x: x.split("/")[-1])
    test_pred_df["fn"] = test_pred_df["fn"].apply(lambda x: f"audio_files/{x}")
    test_pred_df[list(Codes.CODE.keys())] = test_pred
    test_pred_df = test_pred_df[target_cols]
    test_pred_df.to_csv(os.path.join(args.save_path, f"fold-{args.fold}-submission.csv"), index=False)
    print(os.path.join(args.save_path, f"fold-{args.fold}-submission.csv"))

    tta_pred_df = pd.DataFrame({
        "fn" : test_df.fn.values
    })
    tta_pred_df["fn"] = tta_pred_df["fn"].apply(lambda x: x.split("/")[-1])
    tta_pred_df["fn"] = tta_pred_df["fn"].apply(lambda x: f"audio_files/{x}")
    tta_pred_df[list(Codes.CODE.keys())] = tta_pred
    tta_pred_df = tta_pred_df[target_cols]
    tta_pred_df.to_csv(os.path.join(args.save_path, f"tta-fold-{args.fold}-submission.csv"), index=False)
    print(os.path.join(args.save_path, f"tta-fold-{args.fold}-submission.csv"))
    
    oof_pred = Functions.test_epoch(args, model, valid_loader)
    oof_pred_df = pd.DataFrame({
        "fn" : valid_fold.fn.values
    })
    oof_pred_df[list(Codes.CODE.keys())] = oof_pred
    oof_pred_df = oof_pred_df[target_cols]
    oof_pred_df.to_csv(os.path.join(args.save_path, f"oof-fold-{args.fold}.csv"), index=False)
    
if __name__ == "__main__":
    for fold in range(0, 20):
        if fold >= 0:
            main(fold)


Overwriting Run.py


In [ ]:
!python Run.py

2020-11-28 11:19:16.021900: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Train E:0 - Loss5.2150: 100% 69/69 [00:48<00:00,  1.42it/s]
Valid E:0 - Loss:5.1091: 100% 4/4 [00:00<00:00,  4.82it/s]

                Sat Nov 28 11:20:14 2020 

                Fold:17, Epoch:0, lr:0.0002

                Train Loss:5.2150 - LogLoss:5.2150 --- ACC:0.0075 --- F1:0.0075

                Valid Loss:5.1091 - LogLoss:5.1091 --- ACC:0.0085 --- F1:0.0085

        
########## >>>>>>>> Model Improved From inf ----> 5.109109647084766
Train E:1 - Loss5.0779: 100% 69/69 [00:47<00:00,  1.44it/s]
Valid E:1 - Loss:5.0239: 100% 4/4 [00:00<00:00,  4.79it/s]

                Sat Nov 28 11:21:03 2020 

                Fold:17, Epoch:1, lr:0.0004

                Train Loss:5.0779 - LogLoss:5.0779 --- ACC:0.0115 --- F1:0.0115

                Valid Loss:5.0239 - LogLoss:5.0239 --- ACC:0.0213 --- F1:0.0213

        
########## >>>>>>>> Model Im